## Notebook function:
* Connect to API at US Energy Information Agency and download hourly electric grid data *
https://www.eia.gov

API URL:
https://api.eia.gov/v2/electricity/rto/region-data/data/?frequency=hourly&data[0]=value&start=2015-07-01T00&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000

Method:  GET

Series description:
Hourly demand, day-ahead demand forecast, net generation, and interchange by balancing authority. Source: Form EIA-930 Product: Hourly Electric Grid Monitor

API Documentation:  https://www.eia.gov/opendata/documentation.php

*API URL for inventory of operable generators (monthly):*
https://api.eia.gov/v2/electricity/operating-generator-capacity/data/?frequency=monthly&data[0]=county&data[1]=latitude&data[2]=longitude&data[3]=nameplate-capacity-mw&data[4]=net-summer-capacity-mw&data[5]=net-winter-capacity-mw&data[6]=operating-year-month&data[7]=planned-derate-summer-cap-mw&data[8]=planned-derate-year-month&data[9]=planned-retirement-year-month&data[10]=planned-uprate-summer-cap-mw&data[11]=planned-uprate-year-month&start=2018-01&end=2022-12&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000


*API URL for electric power operations for individual power plants (monthly):*
https://api.eia.gov/v2/electricity/facility-fuel/data/?frequency=monthly&data[0]=average-heat-content&data[1]=consumption-for-eg&data[2]=consumption-for-eg-btu&data[3]=generation&data[4]=gross-generation&data[5]=total-consumption&data[6]=total-consumption-btu&start=2018-01&end=2022-12&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000

In [1]:
import requests
import json
import pandas as pd
import time
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta

In [2]:
urlParts_gencap_monthly = {'keyprefix':'https://api.eia.gov/v2/electricity/operating-generator-capacity/data/?',
                        'startprefix':('frequency=monthly&data[0]=county&data[1]=latitude&data[2]=longitude&data[3]'
                                       '=nameplate-capacity-mw&data[4]=net-summer-capacity-mw&data[5]=net-winter-'
                                       'capacity-mw&start='),
                        'endprefix':'&end=',
                        'offsetprefix':'&sort[0][column]=period&sort[0][direction]=desc&offset=', 
                        'lengthprefix':'&length='}

In [3]:
urlParts_genops_monthly = {'keyprefix':'https://api.eia.gov/v2/electricity/facility-fuel/data/?',
                        'startprefix':('frequency=monthly&data[0]=average-heat-content&data[1]='
                                       'consumption-for-eg&data[2]=consumption-for-eg-btu&data[3]='
                                       'generation&data[4]=gross-generation&data[5]=total-'
                                       'consumption&data[6]=total-consumption-btu&start='),
                        'endprefix':'&end=',
                        'offsetprefix':'&sort[0][column]=period&sort[0][direction]=desc&offset=', 
                        'lengthprefix':'&length='}

In [4]:
#retrieves data from EIA api, returns json with response, request, and api metadata
def getEIAdata(api_keystring, url_dict, start, end, offset, length):
    url = (f"{url_dict['keyprefix']}{api_keystring}{url_dict['startprefix']}{start}{url_dict['endprefix']}"
           f"{end}{url_dict['offsetprefix']}{offset}{url_dict['lengthprefix']}{length}")
    #print(url)
    try:
        response = requests.get(url)
    except:
        print(f'no response from {url}')
    try:
        response_json = json.dumps(response.json(), indent=4)
    except:
        print(response)
    return response_json

In [5]:
#returns subset of dictionary containing data items
def extractData(response_dict):
    try:
        data = response_dict['response']['data']
    except KeyError:
        print("Key Error. response_dict:")
        print(response_dict)
    return data

In [6]:
#extracts total number of items that matched API request
def extractTotalRows(response_dict):
    total_rows = response_dict['response']['total']
    return total_rows

In [7]:
#appends data to json file
def saveJSON(json_obj, data_file_path):
    with open(data_file_path, 'a') as fout:
        fout.write(json_obj)

In [8]:
#appends data to csv file
def saveCSV(data_dict, csv_path, header):
    df = pd.DataFrame.from_dict(data_dict)
    with open(csv_path, 'a') as fout:
        df.to_csv(fout, header=header, index=False, lineterminator='\n')

In [12]:
#flow
#api_url_parts = urlParts_gencap_monthly
api_url_parts = urlParts_genops_monthly
api_key = 'DryvLQciETN0UgsSlqTeeQnSfHj8sPif8tfUGKCg'
api_keystring = f"api_key={api_key}&"
row_limit = 5000

start_datetime = '2020-01'
end_datetime = '2023-01'
#start_datetime = f"{start_datetime}T00" #API takes start and end hour in '2023-04-02T00' format
#end_datetime = f"{end_datetime}T00"
api_chill_time = 15

json_path = 'eiadata.json'
csv_path = 'eia.csv'
with open(json_path, 'w') as overwrite:
    pass
with open(csv_path, 'w') as overwrite:
    pass

month_list = []
start = datetime.strptime(start_datetime, "%Y-%m")
end = datetime.strptime(end_datetime, "%Y-%m")
datestepper = start
while datestepper <= end:
    year_month = datestepper.strftime("%Y-%m")
    month_list.append(year_month)
    datestepper += relativedelta(months=+1)
print(month_list)

for month in month_list:
    offset = 0
    start_datetime = month
    end_datetime = month
    response_json = getEIAdata(api_keystring, api_url_parts, start_datetime, end_datetime, offset, row_limit)
    #create dictionary from json object
    d = json.loads(response_json)
    data = extractData(d)
    #saveJSON(response_json, json_path)
    saveCSV(data, csv_path, header=True)
    returned_rows = len(data)
    total_rows = extractTotalRows(d)
    print(f"Total rows:  {total_rows}")
    print(f"downloaded rows:  {returned_rows}")
    call_count = 1
    while offset < total_rows:
        offset += returned_rows
        try:
            response_json = getEIAdata(api_keystring, api_url_parts, start_datetime, end_datetime, offset, row_limit) 
        except UnboundLocalError:
            resume_time = datetime.now() + timedelta(minutes=api_chill_time)
            print(f"API response error. Lurking until {resume_time}")
            time.sleep(60*api_chill_time)
            continue       
        d = json.loads(response_json)
        try:
            data = extractData(d)
        except UnboundLocalError:
            print(f"Error getting data.")
            saveJSON(response_json, json_path)
        #only save first and last chunk to json for examination
        #saveJSON(response_json, json_path)
        returned_rows = len(data)
        print(f"downloaded rows:  {offset + returned_rows}")
        saveCSV(data, csv_path, header=False)
        call_count += 1
        #time.sleep(30)
        if call_count % 20 == 0:
            time.sleep(69)
        #if call_count % 100 == 0:
            #time.sleep(5400)
    #saveJSON(response_json, json_path)
    


['2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12', '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06', '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12', '2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12', '2023-01']
Total rows:  37004
downloaded rows:  5000
downloaded rows:  10000
downloaded rows:  15000
downloaded rows:  20000
downloaded rows:  25000
downloaded rows:  30000
downloaded rows:  35000
downloaded rows:  37004
downloaded rows:  37004
Total rows:  37128
downloaded rows:  5000
downloaded rows:  10000
downloaded rows:  15000
downloaded rows:  20000
downloaded rows:  25000
downloaded rows:  30000
downloaded rows:  35000
downloaded rows:  37128
downloaded rows:  37128
Total rows:  37239
downloaded rows:  5000
downloaded rows:  10000
downloaded rows:  15000
downloaded rows:  20000
downloaded